In [5]:
# ===============================
# 1. Import libraries
# ===============================

import numpy as np
from scipy import signal
import pywt
from pathlib import Path
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from scipy import signal, interpolate

In [6]:
######################################################
# 2. Base values
######################################################

converted_data_folder = Path("./Converted_data")
fs = 2048
no_of_participants = 43
no_of_sessions = 3
no_of_gestures = 5
no_of_trials = 7
# chosen gestures [8]Little Finger Extension, [9]Index Finger Extension, 
# [10]Thumb Finger Extension, [15]Hand Open, [16]Hand Close
gestures = [8, 9, 10, 15, 16]
no_of_gestures = len(gestures)

no_of_features = 7
no_of_wavelet_coeff = 5

In [7]:
######################################################
# 3. Initialize storage arrays
######################################################

a1_forearm = []             # temp list for one gesture's trials
a2_forearm = []             # temp list for one participant's gestures
completeSet_forearm = []    # full dataset for all participants

a1_wrist = []
a2_wrist = []
completeSet_wrist = []

In [8]:
######################################################
# 4. Helper functions
######################################################

def preprocess_emg(signal_data, fs, lowcut=10, highcut=450):
    nyq = fs / 2
    b, a = signal.butter(4, [lowcut/nyq, highcut/nyq], btype='band')
    return signal.filtfilt(b, a, signal_data)

def emg_feature(name, data):
    if name == 'mav': return np.mean(np.abs(data))
    elif name == 'wl': return np.sum(np.abs(np.diff(data)))
    elif name == 'emav': return np.sqrt(np.mean(np.abs(data)))
    elif name == 'ewl': return np.sqrt(np.sum(np.abs(np.diff(data))))
    elif name == 'rms': return np.sqrt(np.mean(np.square(data)))
    elif name == 'zc': return np.sum(np.diff(np.sign(data)) != 0)
    elif name == 'ssc': return np.sum(np.diff(np.sign(np.diff(data))) != 0)
    else: raise ValueError(f"Unknown feature {name}")

def interpolate_nans(arr):
    for ch in range(arr.shape[1]):
        y = arr[:, ch]
        if np.all(np.isnan(y)):
            arr[:, ch] = 0
        else:
            nans = np.isnan(y)
            if np.any(nans):
                x = np.arange(len(y))
                arr[:, ch] = np.interp(x, x[~nans], y[~nans])
    return arr

def extract_features(block):
    n_channels = block.shape[1]
    feat_matrix = np.zeros((n_channels, no_of_features, no_of_wavelet_coeff))
    for ch in range(n_channels):
        coeffs = pywt.wavedec(block[:, ch], 'bior3.3', level=4)
        cA4, cD4, cD3, cD2, cD1 = coeffs
        coeff_list = [cD1, cD2, cD3, cD4, cA4]
        for coeff_idx, coeff_data in enumerate(coeff_list):
            feat_matrix[ch, 0, coeff_idx] = emg_feature('mav', coeff_data)
            feat_matrix[ch, 1, coeff_idx] = emg_feature('wl', coeff_data)
            feat_matrix[ch, 2, coeff_idx] = emg_feature('emav', coeff_data)
            feat_matrix[ch, 3, coeff_idx] = emg_feature('ewl', coeff_data)
            feat_matrix[ch, 4, coeff_idx] = emg_feature('rms', coeff_data)
            feat_matrix[ch, 5, coeff_idx] = emg_feature('zc', coeff_data)
            feat_matrix[ch, 6, coeff_idx] = emg_feature('ssc', coeff_data)
    return feat_matrix


######################################################
# 5. Main loop
######################################################
FV_forearm = [[None for _ in range(no_of_gestures)] for _ in range(no_of_participants)]
FV_wrist   = [[None for _ in range(no_of_gestures)] for _ in range(no_of_participants)]

count = 0

for session_num in range(1, no_of_sessions+1):
    for participant_num in range(1, no_of_participants+1):
        file_path = converted_data_folder / f"Session{session_num}_Converted" / f"session{session_num}_participant{participant_num}.npz"
        if not file_path.exists():
            print(f"Missing: {file_path}. Breaking the cycle.")
            break

        data = np.load(file_path, allow_pickle=True)
        forearm_data = data['forearmData']
        wrist_data   = data['wristData']

        for g_idx, gesture in enumerate(gestures):
            all_forearm = []
            all_wrist   = []
            for trial_num in range(no_of_trials):
                seg_forearm = forearm_data[trial_num, gesture-1]
                seg_wrist   = wrist_data[trial_num, gesture-1]
                if seg_forearm is None or seg_wrist is None:
                    continue
                all_forearm.append(seg_forearm)
                all_wrist.append(seg_wrist)

            if not all_forearm or not all_wrist:
                continue

            forearm_concat = np.vstack(all_forearm)
            wrist_concat   = np.vstack(all_wrist)

            forearm_concat = forearm_concat - np.mean(forearm_concat, axis=0, keepdims=True)
            wrist_concat   = wrist_concat - np.mean(wrist_concat, axis=0, keepdims=True)

            forearm_concat = interpolate_nans(forearm_concat)
            wrist_concat   = interpolate_nans(wrist_concat)

            preproc_forearm = np.zeros_like(forearm_concat)
            preproc_wrist   = np.zeros_like(wrist_concat)
            for ch in range(forearm_concat.shape[1]):
                preproc_forearm[:, ch] = preprocess_emg(forearm_concat[:, ch], fs)
            for ch in range(wrist_concat.shape[1]):
                preproc_wrist[:, ch] = preprocess_emg(wrist_concat[:, ch], fs)

            FV_forearm[participant_num-1][g_idx] = extract_features(preproc_forearm)
            FV_wrist[participant_num-1][g_idx]   = extract_features(preproc_wrist)

            count += 1
            print(f"Processed: {count} / {no_of_participants*no_of_sessions*no_of_gestures}")

np.savez_compressed("Feature_vector.npz", FV_forearm=FV_forearm, FV_wrist=FV_wrist)
print("Feature vectors saved successfully.")

Processed: 1 / 645
Processed: 2 / 645
Processed: 3 / 645
Processed: 4 / 645
Processed: 5 / 645
Processed: 6 / 645
Processed: 7 / 645
Processed: 8 / 645
Processed: 9 / 645
Processed: 10 / 645
Processed: 11 / 645
Processed: 12 / 645
Processed: 13 / 645
Processed: 14 / 645
Processed: 15 / 645
Processed: 16 / 645
Processed: 17 / 645
Processed: 18 / 645
Processed: 19 / 645
Processed: 20 / 645
Processed: 21 / 645
Processed: 22 / 645
Processed: 23 / 645
Processed: 24 / 645
Processed: 25 / 645
Processed: 26 / 645
Processed: 27 / 645
Processed: 28 / 645
Processed: 29 / 645
Processed: 30 / 645
Processed: 31 / 645
Processed: 32 / 645
Processed: 33 / 645
Processed: 34 / 645
Processed: 35 / 645
Processed: 36 / 645
Processed: 37 / 645
Processed: 38 / 645
Processed: 39 / 645
Processed: 40 / 645
Processed: 41 / 645
Processed: 42 / 645
Processed: 43 / 645
Processed: 44 / 645
Processed: 45 / 645
Processed: 46 / 645
Processed: 47 / 645
Processed: 48 / 645
Processed: 49 / 645
Processed: 50 / 645
Processed